<a href="https://colab.research.google.com/github/jenny550/mit_futuremakers_create-a-thon_2023/blob/main/wildfires.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Wildfire Image Detection

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import PIL
from PIL import Image
import os

#Model Architecture

In [ ]:
#history =  model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs,
                  #   validation_data = #Enter Code ,
                    # verbose=1)

In [ ]:
#loading images into arrays

train_fire_arr = [np.array( Image.open(f'/content/training/fire/{x}').resize((128,128)) ) for x in os.listdir('/content/training/fire')] #training fie imgs
train_noFire_arr = [np.array(Image.open(f'/content/training/noFire/{x}', mode='r').resize((128,128)) ) for x in os.listdir('/content/training/noFire')] #training noFire imgs

test_fire_arr = [np.array(Image.open(f'/content/testing/fire/{x}', mode='r').resize((128,128)) ) for x in os.listdir('/content/testing/fire')] #testing fire imgs
test_noFire_arr = [np.array(Image.open(f'/content/testing/noFire/{x}', mode='r').resize((128,128)) ) for x in os.listdir('/content/testing/noFire')] #testing noFire imgs

train = np.array(train_fire_arr + train_noFire_arr)
print('train: fire length: ', len(train_fire_arr))
print('train: noFire length: ', len(train_noFire_arr))
print('train: fire + noFire length: ', len(train))

test = np.array(test_fire_arr + test_noFire_arr)
print('test: fire length: ', len(test_fire_arr))
print('test: noFire length: ', len(test_noFire_arr))
print('test: fire + noFire length: ', len(test))

#print(train_fire_arr[0])
#plt.imshow(train[1800])

test[0].shape # each img is 250 x 250

In [ ]:
# x_train = images of fires
#labels = {0: no fire, 1.0: fire}
#y_train = all labels of fire, hint: {1.0}

x_train, y_train = train, np.concatenate((np.ones(len(train_fire_arr)), np.zeros(len(train_noFire_arr))), axis = 0)
x_test, y_test = test, np.concatenate((np.ones(len(test_fire_arr)), np.zeros(len(test_noFire_arr))), axis = 0)
idx = -1

x_train = x_train / 255.0
x_test = x_test / 255.0

print(y_train[idx])
print(x_train[idx])
plt.imshow(x_train[idx])

In [ ]:
!rm -rf `find -type d -name .ipynb_checkpoints`

In [ ]:
train = ImageDataGenerator(rescale=1/255) # conguring training dataset frame, pixel / 255
test = ImageDataGenerator(rescale=1/255) # conguring training dataset frame, pixel / 255

train_dataset = train.flow_from_directory("/content/training",
                                          target_size=(128,128),
                                          batch_size = 32,
                                          class_mode='binary') # further configuring dataset frame, already loaded with image contents

test_dataset = test.flow_from_directory("/content/testing",
                                          target_size=(128,128),
                                          batch_size =32,
                                          class_mode = 'binary')  # further configuring dataset frame, already loaded with image contents

In [ ]:
train_dataset.class_indices #print out class indicies: 0 for {fire}, 1 for {no fire }

In [ ]:
#Copied from CNN Module just to get us started
model = keras.Sequential()
model.add(keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(128,128,3)))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(64,(3,3),activation='relu'))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(128,(3,3),activation='relu'))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(128,(3,3),activation='relu'))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(512,activation='relu'))
model.add(keras.layers.Dense(1,activation='sigmoid'))

#model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
#model.summary()

In [ ]:
#y_train.resize(1832, 1)
#print(y_train[2])
model.fit(train_dataset, epochs = 10)

In [ ]:
#predictions
def predict(img):
  plt.imshow(img)
  X = np.expand_dims(img, axis = 0)
  predVal = model.predict(X)
  print(predVal)

  if predVal == 1:
    plt.xlabel('No Fire', fontsize = 34)
  else:
    plt.xlabel('Fire', fontsize = 34)

In [ ]:
#test_fire_arr
#test_noFire_arr
predict(test_noFire_arr[-15])

In [ ]:
#predict whole testing set
predictions = np.round(model.predict(test_dataset))
print(predictions)
for img in test_dataset:
  plt.imshow(img)